In [1]:
import os
import pandas as pd
import numpy as np
import h5py
from collections import Counter
try:
    import cPickle as pickle
except:
    import pickle

import sys
sys.path.append(os.path.abspath('../src'))

from enrichment_helper import *

from statsmodels.sandbox.stats.multicomp import multipletests
from statsmodels.stats.proportion import proportion_confint
import scipy.stats as stats

import sqlite3
with open('../src/ensembl2genename_GRCh38.85.p', 'rb') as reader:
    gid2gn = pickle.load(reader)   
    
  

In [2]:
selected_phenotype = 'IBD.EUR.Crohns_Disease'
lincs_file = 'CD_signatures/CD_lincs_l1000.p'
fn_out = 'CD_targets.csv'
mgi_search_criteria = lambda x: ('bowel' in x.lower()) or ('crohn' in x.lower())
creed_search_criteria = lambda x: ('bowel' in x.lower()) or ('crohn' in x.lower())

In [3]:
## MGI
hg2mg, mg, mg2pheno, pheno, pheno2name = load_mgi('../../../project_wrapup/GSK_project/GWAS_component/data/MGI/')  

df = pd.read_csv('../../../project_wrapup/GSK_project/GWAS_component/data/MGI/MGI_DO.rpt', sep = '\t')

dis2pheno = {}
for d, m in zip(df['DO Disease Name'], df['Mouse MGI ID']):
    if mgi_search_criteria(d) and m != None:
        if m in mg:
            print d, m, m in mg, len(mg2pheno[m])
            dis2pheno[m] = set(mg2pheno[m])
        #else:
            #print d, m, m in mg

Crohn's disease MGI:1335106 True 77
Crohn's disease MGI:105311 True 11
inflammatory bowel disease MGI:102672 True 96
inflammatory bowel disease MGI:104798 True 221
inflammatory bowel disease MGI:103038 True 173
inflammatory bowel disease MGI:109495 True 45
inflammatory bowel disease MGI:1339364 True 44
inflammatory bowel disease MGI:96548 True 104
inflammatory bowel disease MGI:96537 True 188
inflammatory bowel disease MGI:88337 True 57
inflammatory bowel disease MGI:96549 True 38
inflammatory bowel disease 1 MGI:2429397 True 37
inflammatory bowel disease 10 MGI:1924290 True 40
inflammatory bowel disease 12 MGI:95772 True 78
inflammatory bowel disease 13 MGI:97570 True 40
inflammatory bowel disease 16 MGI:2180140 True 13


In [4]:
with open('./gcomp_drive_genes.p', 'rb') as reader:
    drivers = set([gid2gn[item[0]] for s, item in pickle.load(reader)[selected_phenotype]])

In [5]:
#!/usr/bin/env python
import pandas
import sqlite3

connection = sqlite3.connect("../../data/gwas_g2p/metaxcan_results_p/metaxcan_results_v1.5.db")

query = 'SELECT g.gene_name, m.zscore, m.n_snps_used, m.n_snps_model, p.tag as phenotype, t.tissue as tissue, g.gene ' 
query += ' FROM gene AS g INNER JOIN metaxcan_result AS m ON g.id = m.gene_id' 
query += ' INNER JOIN tissue AS t ON t.id = m.tissue_id  INNER JOIN pheno AS p ON p.id = m.pheno_id'
query += ' WHERE p.tag = \'%s\'' % selected_phenotype

pi = pandas.read_sql_query(query, connection)
pi = pi[pi.tissue != 'DGN_WB']


In [6]:
from scipy.stats import norm 
import statsmodels.stats.multitest as multi

pvals = norm.sf(np.abs(pi.zscore)) * 2
_, res_adjp, _, _ = multi.multipletests(pvals)


In [7]:
sgenes = set(pi.gene_name[res_adjp < 0.05])
print len(sgenes)

150


In [8]:
mgi_res_meta = mgi_match(sgenes, hg2mg, mg2pheno, dis2pheno, pheno, alpha_sig = (0.05) / float(len(dis2pheno)))
mgi_hit_meta = set([g for g, c in mgi_res_meta.items()])
n1 = len(mgi_hit_meta)
n0 = len([g for g in sgenes if g in hg2mg])
print(n1, n0, float(n1) / float(n0), proportion_confint(n1, n0))

(50, 136, 0.36764705882352944, (0.28661181256202545, 0.44868230508503343))


In [9]:
mgi_res_gc = mgi_match(drivers, hg2mg, mg2pheno, dis2pheno, pheno, alpha_sig = (0.05) / float(len(dis2pheno)))
mgi_hit_gc = set([g for g, c in mgi_res_gc.items()])
n1 = len(mgi_hit_gc)
n0 = len([g for g in drivers if g in hg2mg])
print(n1, n0, float(n1) / float(n0), proportion_confint(n1, n0))

(15, 46, 0.32608695652173914, (0.1906186131351622, 0.46155529990831606))


In [10]:
s = ''
for i, g in enumerate(drivers | sgenes):
    if (i % 8) == 0:
        print(s)
        s = ''
    s += '\'%s\', ' % g


'SLC6A1', 'CTD-2330K9.3', 'SCG3', 'PGAP3', 'MAMSTR', 'MST1', 'P4HA2', 'P4HA3', 
'ARFRP1', 'IL27', 'NKX2-3', 'TYK2', 'MICB', 'MICA', 'IL1RL1', 'MAP3K9', 
'DDX39B', 'TMEM182', 'PTGER4', 'GSDMB', 'EIF3C', 'CDC42SE2', 'CLN3', 'FAM110D', 
'MTMR7', 'RERGL', 'FAM166A', 'EIF3CL', 'WDR6', 'NCKIPSD', 'APOBR', 'WNK4', 
'CHSY3', 'LDB3', 'PALM', 'PROM2', 'SIGLEC10', 'BRD7', 'IP6K2', 'HRC', 
'ORM1', 'INPP5E', 'LST1', 'HCN3', 'MST1R', 'TRAIP', 'RAI14', 'ZGPAT', 
'CYLD', 'LAT', 'DDAH2', 'ERAP2', 'ACTA1', 'ATP2A1', 'STMN3', 'GBA', 
'ANO1', 'RIT1', 'C2orf82', 'UBE2L3', 'SEC16A', 'OVGP1', 'RTEL1', 'CCDC116', 
'TNXB', 'THBS3', 'NID2', 'CARD9', 'LCN10', 'C4B', 'HLA-DQB2', 'CARD6', 
'TRPM2', 'PDLIM4', 'ALPK3', 'CISD1', 'PDLIM3', 'NUPR1', 'FAM212A', 'UBA7', 
'DAG1', 'PAPD5', 'SBK1', 'IRGM', 'PRDM16', 'IRF1', 'CDHR4', 'RBPMS', 
'C6orf47', 'NPIPB6', 'TTC33', 'SLC22A5', 'RBM22', 'GPX4', 'RBM20', 'FNIP1', 
'SNX20', 'RNF123', 'C1orf141', 'IL12RB2', 'SAG', 'SDCCAG3', 'SMAD3', 'CDSN', 
'RNASET2', 'CCDC101', 'PKLR'

In [11]:
## Lincs 1000
df0 = pd.read_csv('../../../data/CREED/Disease_entire_exp_landmark.tsv.gz', sep = '\t')
df1 = pd.read_csv('../../../data/CREED/Disease_signatures_info_data.tsv', sep = '\t')

ind = [d0 for d0, d1 in zip(df1.id, df1.disease_name) if creed_search_criteria(d1)]
dis = df0[ind].values.T
dis = dis / np.linalg.norm(dis, axis = 1, keepdims=True)
print dis.shape

(5, 978)


In [12]:
with open('CD_signatures/REF_lincs_l1000.p', 'rb') as f:
    res = pickle.load(f)

cd_ref = np.vstack([item[u'chdirLm'] for item in res]) 
cd_ref /= (np.sqrt(np.sum(cd_ref **2, axis = 1)) + 1e-5)[:, np.newaxis]

with open(lincs_file, 'rb') as f:
    res_shrna = pickle.load(f)

In [13]:
lincs_res_meta = lincs_match(sgenes, res_shrna, dis, cd_ref, alpha_sig = 0.05)
lincs_hit_meta = set([g for g, item in lincs_res_meta.items() if item[0] > 0])
n1 = len(lincs_hit_meta)
n0 = len(lincs_res_meta)
print(n1, n0, float(n1) / float(n0), proportion_confint(n1, n0))

(18, 74, 0.24324324324324326, (0.14549008953641684, 0.34099639695006967))


In [14]:
lincs_res_gc = lincs_match(drivers, res_shrna, dis, cd_ref, alpha_sig = 0.05)
lincs_hit_gc = set([g for g, item in lincs_res_gc .items() if item[0] > 0])
n1 = len(lincs_hit_gc)
n0 = len(lincs_res_gc)
print(n1, n0, float(n1) / float(n0), proportion_confint(n1, n0))

(2, 14, 0.14285714285714285, (-0.04044251670493995, 0.3261568024192256))


In [15]:
id2gene = sorted( drivers | sgenes)

In [16]:
mgi_genes = set([g for g in (drivers|sgenes) if g in hg2mg])
mgi_hits = (mgi_hit_gc | mgi_hit_meta)
lincs_genes =  (set(lincs_res_gc) | set(lincs_res_meta))
lincs_hits = (lincs_hit_gc | lincs_hit_meta)

In [17]:
df_res = []
for item in [drivers, sgenes, lincs_hits, lincs_genes, mgi_hits, mgi_genes]:
    vec = [int(g in item) for g in id2gene]
    df_res.append(vec)
df_res = pd.DataFrame(df_res).T

In [18]:
df_res.columns = ['genes_selected_by_GCA+BNs', 'genes_selected_by_metaxcan', 
                  'LINCS_hit', 'genes_found_in_LINCS', 'MGI_hit', 'genes_found_in_MGI']
df_res.index = id2gene

df_res.to_csv(fn_out)